In [8]:
!pip install --upgrade pip
!pip -q install langchain openai tiktoken chromadb pypdf sentence_transformers InstructorEmbedding
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip -q install sentencepiece Xformers einops
!pip -q install unstructured pandoc

In [9]:
!nvidia-smi

Wed Aug  2 11:15:13 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.182.03   Driver Version: 470.182.03   CUDA Version: N/A      |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            On   | 00000000:00:1E.0 Off |                    0 |
| N/A   29C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
!pip show langchain

Name: langchain
Version: 0.0.249
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /home/cdsw/.local/lib/python3.9/site-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [15]:
!pip install accelerate

# LangChain multi-doc retriever with ChromaDB

***New Points***
- Multiple Files - Epubs
- ChromaDB -
- Source info
- Stable Vicuna
- HuggingFace Embeddings


In [19]:
import torch
import transformers
from transformers import GenerationConfig, pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig

import bitsandbytes as bnb

In [20]:
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline

tokenizer = LlamaTokenizer.from_pretrained("eachadea/vicuna-7b-1.1")

model = LlamaForCausalLM.from_pretrained("eachadea/vicuna-7b-1.1",
                                              load_in_8bit=True,
                                              device_map='auto',
                                              torch_dtype=torch.float16,
                                              low_cpu_mem_usage=True
                                              )

ImportError: Using `load_in_8bit=True` requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes `pip install -i https://test.pypi.org/simple/ bitsandbytes` or pip install bitsandbytes` 

In [5]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline
import torch

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=2048,
    temperature=0,
    top_p=0.95,
    repetition_penalty=1.15
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [6]:
from langchain.llms import HuggingFacePipeline
import torch

local_llm = HuggingFacePipeline(pipeline=pipe)

In [7]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [8]:
local_llm('How are you today?')

"\nI'm doing well, thank you for asking! Just a little stressed out about the move and all that comes with it. But I'll get through it."

## Setting up LangChain


In [9]:
import os



In [10]:
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader


## Load Epub

In [13]:
from langchain.document_loaders import UnstructuredEPubLoader

#epub_loader = UnstructuredEPubLoader("/content/Running Lean_ Iterate from Plan A to a Plan That Works (Lean Series) - Maurya, Ash.epub") #, mode="elements"
epub_loader = UnstructuredEPubLoader("/home/cdsw/sherlock_holmes.epub") #, mode="elements"


epub_data = epub_loader.load()

In [14]:
from pypandoc.pandoc_download import download_pandoc

In [15]:
# see the documentation how to customize the installation path
# but be aware that you then need to include it in the `PATH`
download_pandoc()

In [16]:
len(epub_data)

1

In [17]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts_01 = text_splitter.split_documents(documents)
texts_02 = text_splitter.split_documents(epub_data)

In [18]:
len(texts_01), len(texts_02)

(112, 815)

In [19]:
texts = texts_01 + texts_02

In [20]:
len(texts)

927

In [21]:
texts[370]

Document(page_content='“His son, you see, had grown up, and so had my girl, and as I was\nknown to be in weak health, it seemed a fine stroke to him that his lad\nshould step into the whole property. But there I was firm. I would not\nhave his cursed stock mixed with mine; not that I had any dislike to the\nlad, but his blood was in him, and that was enough. I stood firm.\nMcCarthy threatened. I braved him to do his worst. We were to meet at\nthe pool midway between our houses to talk it over.', metadata={'source': '/home/cdsw/sherlock_holmes.epub'})

## HF Embeddings

In [22]:
from langchain.embeddings import HuggingFaceEmbeddings, SentenceTransformerEmbeddings

model_name = "intfloat/e5-large-v2"

hf = HuggingFaceEmbeddings(model_name=model_name)

In [23]:
hf

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='intfloat/e5-large-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

## create the DB

In [24]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = './db'

## Here is the nmew embeddings being used
embedding = hf #instructor_embeddings

vectordb = Chroma.from_documents(documents=texts,
                                 embedding=embedding,
                                 persist_directory=persist_directory)

## Make a retriever

In [25]:
retriever = vectordb.as_retriever()

In [47]:
docs = retriever.get_relevant_documents("who is Irene Adler?")

In [48]:
len(docs)

2

In [49]:
docs[0]

Document(page_content='his mental results. Grit in a sensitive instrument, or a crack in one of\nhis own high-power lenses, would not be more disturbing than a strong\nemotion in a nature such as his. And yet there was but one woman to him,\nand that woman was the late Irene Adler, of dubious and questionable\nmemory.', metadata={'source': '/home/cdsw/sherlock_holmes.epub'})

In [50]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [51]:
retriever.search_type

'similarity'

In [52]:
retriever.search_kwargs

{'k': 2}

## Make a chain

In [53]:
# create the chain to answer questions

qa_chain = RetrievalQA.from_chain_type(llm=local_llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)



In [54]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [ ]:
# print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

In [55]:
qa_chain.combine_documents_chain.llm_chain.prompt.template = '''
Your name is Sherlock Holmes.
Use the following pieces of context to answer the users question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Always answer from the perspective of being Sherlock Holmes.
----------------
{context}

Question: {question}
Helpful Answer:'''

In [56]:
def trim_string(input_string):
    input_string = str(input_string)
    trim_index = input_string.find("### Human:")
    if trim_index != -1:  # If the phrase is found
        return input_string[:trim_index]
    else:
        return input_string  # If the phrase isn't found, return the original string

In [57]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

def process_llm_response(llm_response):
    temp_resp = wrap_text_preserve_newlines(llm_response['result'])
    temp_resp = trim_string(temp_resp)
    print(temp_resp)
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [63]:
# full example
query = "In which case was Sherlock holmes involved with Royalty?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 The King of Scandinavia


Sources:
/home/cdsw/sherlock_holmes.epub
/home/cdsw/sherlock_holmes.epub


In [66]:
# break it down
query = "How did Sherlock holmes know that Dr Watson was earlier in the army? "
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Sherlock Holmes observed that Dr Watson had a scar above his wrist, which indicated that he had once injured
his arm while serving in the army.


Sources:
/home/cdsw/sherlock_holmes.epub
/home/cdsw/sherlock_holmes.epub


In [70]:
query = "Were Dr Watson and Sherlock Holmes friends?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 Yes, they were friends.


Sources:
/home/cdsw/sherlock_holmes.epub
/home/cdsw/sherlock_holmes.epub


In [71]:
query = "Were Dr Moriarty and Sherlock Holmes friends?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 No, Dr Moriarty and Sherlock Holmes were not friends.


Sources:
/home/cdsw/sherlock_holmes.epub
/home/cdsw/sherlock_holmes.epub


In [72]:
query = "What was the hardest case for Sherlock holmes to solve?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 The hardest case for Sherlock Holmes to solve was the one where he had to determine what caused a man to
become insane after eating a hunchbacked man's heart.


Sources:
/home/cdsw/sherlock_holmes.epub
/home/cdsw/sherlock_holmes.epub


In [73]:
query = "Which are some of the most popular cases by Sherlock holmes"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 Some of the most popular cases by Sherlock Holmes include "A Study in Scarlet," "The Hound of the
Baskervilles," "The Sign of Four," and "The Adventure of the Speckled Band."


Sources:
/home/cdsw/sherlock_holmes.epub
/home/cdsw/sherlock_holmes.epub


In [75]:
query = "what was the villain in the Hound of Baskervilles?"
llm_response = qa_chain(query)
process_llm_response(llm_response)


 The villain in "The Hound of the Baskervilles" by Sir Arthur Conan Doyle is Sir Charles Baskerville.


Sources:
/home/cdsw/sherlock_holmes.epub
/home/cdsw/sherlock_holmes.epub


In [78]:
query = "who were the four characters in the Sign of Four who took the oath?"
llm_response = qa_chain(query)
output = process_llm_response(llm_response)
trim_string(output)

 The four characters in "The Sign of Four" who took the oath were Dr. Grimesby Roylott, Sir George Burnwell,
Lady Carmichael, and her daughter Helen.


Sources:
/home/cdsw/sherlock_holmes.epub
/home/cdsw/sherlock_holmes.epub


'None'